# <b> Buy and Hold </b>

Example showing the backtesting of a Buy and Hold strategy.

When the strategy starts, it will buy 50% of AAPL and 50% of GOOG.
As soon as the backtest completes, it will return a dataframe 'result' containing the results, which can be used to visualize the equity's evolution and other metrics, like when buy or sell orders are placed.

## <b> Import Libraries </b>

In [3]:
# Import this library
from nbacktest import Backtest, Strategy

# Import other libraries
import yfinance as yf
import warnings

# Ignore warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## <b> Create Strategy </b>

In [4]:
class TestStrategy (Strategy):

    def on_start(self):

        print("Strategy started!")
        
        quantity1 = int((self.balance/2)/self.prices["AAPL"])
        quantity2 = int((self.balance/2)/self.prices["GOOG"])

        order1 = self.buy("AAPL", quantity1)
        order2 = self.buy("GOOG", quantity2)

        trade1 = self.create_trade(orders=[order1, order2], notes="Buy and Hold Trade")

        print("Bought %s AAPL and %s GOOG" % (quantity1, quantity2))


    def on_end(self):
        """"
        This function is called when the backtest of all candles is finished
        """
        print("Strategy finished!")


    def next(self):
        """"
        This function is called every candle (iteration)
        """
        #print("Backtest iteration:", self.iteration)
        pass


## <b> Download OHLC data from Yahoo Finance </b>

In [5]:
universe = ["AAPL", "GOOG"] # List containing the tickers of all the assets you are backtesting

df_ohlc = yf.download(universe, start="2018-01-01", end="2023-01-01", interval="1d") # Download df_ohlc data from Yahoo Finance

print(df_ohlc.shape)
df_ohlc.tail()

[*********************100%***********************]  2 of 2 completed

(1259, 10)


Price            Close                   High                    Low  \
Ticker            AAPL       GOOG        AAPL       GOOG        AAPL   
Date                                                                   
2022-12-23  129.900284  89.141418  130.451959  89.429260  127.713276   
2022-12-27  128.097473  87.275414  129.456968  88.833725  126.806945   
2022-12-28  124.166801  85.816368  129.082638  87.861030  123.999330   
2022-12-29  127.683723  88.287819  128.540789  88.699730  125.831667   
2022-12-30  127.998947  88.069473  128.018654  88.168727  125.536110   

Price                        Open               Volume            
Ticker           GOOG        AAPL       GOOG      AAPL      GOOG  
Date                                                              
2022-12-23  86.967727  128.974252  86.967727  63814900  17815000  
2022-12-27  86.883357  129.427415  88.645138  69007800  15470900  
2022-12-28  85.727042  127.742850  86.848627  85438400  17879600  
2022-12-29  86.342411  126.087797  86.382114  75703700  18280700  
2022-12-30  86.382124  126.501549  86.714629  77034200  19190300

## <b> Run Backtest </b>

In [6]:
bt = Backtest(data=df_ohlc,
              universe=universe,
              strategy_class=TestStrategy,
              price_column="Close",
              cash=100_000
             )

result = bt.run()

result.head()

Strategy started!
Bought 1239 AAPL and 946 GOOG
Strategy finished!


,BALANCE,POSITIONS_FILLED_TOTAL,POSITIONS_UNFILLED_TOTAL,EQUITY
ITERATION,,,,
2018-01-02,16.904774,99983.095226,0,100000.000000
2018-01-03,16.904774,100795.039742,0,100811.944515
2018-01-04,16.904774,101211.196243,0,101228.101017
2018-01-05,16.904774,102526.014412,0,102542.919186
2018-01-08,16.904774,102558.532574,0,102575.437347


## <b> Backtest Statistics </b>

In [7]:
print("---------------------------")
print("Final balance: %.2f" % bt.strategy.balance)
print("Final equity: %.2f" % bt.strategy.equity)
print("---------------------------")

---------------------------
Final balance: 16.90
Final equity: 241921.32
---------------------------


In [8]:
bt.get_trade_statistics()

{'n_won': 1,
 'n_lost': 0,
 'n_total': 1,
 'win_rate': 1.0,
 'mean_profit': 141921.3219947815,
 'median_profit': 141921.3219947815,
 'mean_profit_win': 141921.3219947815,
 'mean_profit_loss': nan,
 'median_profit_win': 141921.3219947815,
 'median_profit_loss': nan,
 'std_profit': nan,
 'mad_profit': 0.0,
 'skewness': nan,
 'kurtosis': nan,
 'mean_median_gap': 0.0,
 'mean_profit_over_vol': nan,
 'median_profit_over_vol': nan}

In [9]:
bt.get_equity_statistics_dollar()

{'total_profit': 141921.3219947815,
 'mean_profit': 112.81504133130484,
 'median_profit': 256.2090492248535,
 'std_profit': 4002.753357499658,
 'mad_profit': 2424.439759176415,
 'skewness': -0.19350086115521806,
 'kurtosis': 3.0568943057879236,
 'mean_median_gap': -143.3940078935487,
 'mean_profit_over_vol': 0.028184359928130915,
 'median_profit_over_vol': 0.0640082029398124,
 'max_drawdown': -122839.04663848877}

In [10]:
bt.get_equity_statistics_return()

{'total_return': 1.4192132199478151,
 'mean_return': 0.0008838017029635996,
 'median_return': 0.0016635665753590434,
 'std_return': 0.01902778181735057,
 'mad_return': 0.014074732554664288,
 'skewness': -0.1545732086046939,
 'kurtosis': 4.071150021118391,
 'mean_median_gap': -0.0007797648723954438,
 'mean_return_over_vol': 0.046447962849652864,
 'median_return_over_vol': 0.08742829780831902,
 'max_drawdown': -0.34324001569572316}

In [11]:
bt.tradebook

,ID,STATUS,NOTES,PNL,CREATED_ITERATION,CLOSED_ITERATION,REASON_CLOSED,POSITIONS_FILLED,POSITIONS_FILLED_TOTAL,POSITIONS_UNFILLED,POSITIONS_UNFILLED_TOTAL,STOP_LOSS,TAKE_PROFIT,MAX_AGE,AGE
0,a6bc3bb7-f6df-4d4c-8db3-a7a8f14cdceb,OPEN,Buy and Hold Trade,141921.321995,2018-01-02,None,None,"{'AAPL': {'quantity': 1239, 'value': 158590.69...",241904.417221,{},0,None,None,None,1823 days


In [12]:
bt.orderbook

,ID,TICKER,ACTION,STATUS,TRADE_ID,REQUESTED_ITERATION,REQUESTED_QUANTITY,REQUESTED_PRICE,REQUESTED_FEE,REQUESTED_GROSS_TOTAL,REQUESTED_TOTAL,FILLED_ITERATION,FILLED_QUANTITY,FILLED_PRICE,FILLED_FEE,FILLED_GROSS_TOTAL,FILLED_TOTAL
0,5a595c63-2f38-40e9-be05-fc5c5314653c,AAPL,BUY,FILLED,a6bc3bb7-f6df-4d4c-8db3-a7a8f14cdceb,2018-01-02,1239,40.341892,0.0,49983.604488,49983.604488,2018-01-02,1239,40.341892,0.0,-49983.604488,-49983.604488
1,b0baf05e-4e3b-4806-af0a-2188df7009c5,GOOG,BUY,FILLED,a6bc3bb7-f6df-4d4c-8db3-a7a8f14cdceb,2018-01-02,946,52.853584,0.0,49999.490738,49999.490738,2018-01-02,946,52.853584,0.0,-49999.490738,-49999.490738


In [ ]:
bt.strategy.positions_filled

AttributeError: 'TestStrategy' object has no attribute 'filled_positions'

In [24]:
bt.strategy.positions_unfilled

{}

In [21]:
bt.strategy.positions_filled

{'AAPL': {'quantity': 1239, 'value': np.float64(158590.69551086426)},
 'GOOG': {'quantity': 946, 'value': np.float64(83313.72171020508)}}